# Pipeline tp predict popularity of a song on spotify based on 2022 data

In [ ]:
# load the data as df
df = loaddata(r'C:\Users\vanes\Git_repositories\HW3\sample_diabetes_mellitus_data.csv')

In [ ]:
# drop unnecessary columns: track_id and Unnamed: 0
df = drop(data = df, columns_to_drop=['track_id', 'Unnamed: 0']).drop_columns()

In [ ]:
# transform column duration_ms to a column where the duration is in minutes
df = transform_columns(df, 'duration_ms').transform_ms_to_minutes()